### Current Headways

In [1]:
from shapely import wkb
import pyproj
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sshtunnel 
import pandas as pd
import psycopg2.extras
import pyproj
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points


### Calculate distance between points of each line

In [2]:
!pip3 install shapely --user
!pip3 install pyproj --user
!pip3 install geopy --user


In [3]:
#connect to database and get the trips and planned stops for a line
def get_data_from_lines():
    
    #SQL statement
    sql = """Select *
    from actu_lines"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [4]:
#connect to database and get the trips and planned stops for a line
def get_data_from_points():
    
    #SQL statement
    sql = """Select *
    from actu_stops"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [5]:
lines=get_data_from_lines()
stops=get_data_from_points()

In [6]:
import geopy.distance

def calculate_distance_between_each_stop(lines, stops):
    result=[]
    geod = pyproj.Geod(ellps='WGS84')
    
    for index, i in lines.iterrows():
        filter_stops=stops[(stops.code_ligne==str(i["ligne"]))&(stops.variante==i["variante"])]
        filter_stops=filter_stops.sort_values(by=["succession"])
        b=bytes(i["wkb_geometry"])
        merged = wkb.loads(b)
        for j in range(0,len(filter_stops)-1):
            p1 = wkb.loads(bytes(filter_stops.iloc[j]["wkb_geometry"]))
            p2 = wkb.loads(bytes(filter_stops.iloc[j+1]["wkb_geometry"]))
            point1=nearest_points(p1,merged)[0]
            point2=nearest_points(p2,merged)[0]
            
            distance_along_track = abs( merged.project(p1) - merged.project(p2) )
            coords_1 = (point1.x, point1.y)
            coords_2 = (point2.x, point2.y)

            #distance= geopy.distance.geodesic(coords_1, coords_2).km
            distance= point1.distance(point2)
            result.append( {
                                            'code_ligne': filter_stops.iloc[j]["code_ligne"],
                                            'variante': filter_stops.iloc[j]["variante"],
                                            'stop_1': filter_stops.iloc[j]["stop_id"],
                                            'stop_2': filter_stops.iloc[j+1]["stop_id"],
                                            'distance': distance_along_track,
                                            'distance_m': distance

                            })
    df_methods = pd.DataFrame(result)
    df_methods.to_csv("./realability/distance_stops.csv")
    return df_methods


In [7]:
df_methods=calculate_distance_between_each_stop(lines, stops)

third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r mu

In [8]:
df_distances=pd.read_csv("./realability/distance_stops.csv")
